## Replicating Generators

The below represents the parent class for our custom generator object.

In [1]:
# Everywhere we used a generator, we'll instead use a child of this class.

#this is our custom iterator, not a subclass of Iterator class!
#this is our parent class
class Iterator:     
    def hasNext(self):
        """Intended behavior: returns whether there is a next element."""
        raise NotImplementedError
    
    def next(self):
        """Intended behavior: returns next element.
        Illegal to call when hasNext() returns False!"""
        raise NotImplementedError
    
    # Bonus: we use yet another magic method to build a native Python iterator
    # from one of our hand-crafted ones.

    #this is the bit to create an actual Python iterator!
    def __iter__(self):
        while self.hasNext():
            yield self.next()

Duplicating the generation of all positive integers

In [2]:
class AllPositiveIntegers(Iterator):
    def __init__(self):
        self.i = 1 # Attribute to hold next integer we should generate

    def hasNext(self):
        return True # Infinite streams always have next values!

    def next(self):
        result = self.i
        self.i += 1
        return result

In [3]:
api = AllPositiveIntegers()
print(api.next())
print(api.next())
print(api.next())

1
2
3


first-n functionality to generate the first `n` items in a stream

In [4]:
class FirstN(Iterator):
    def __init__(self, child, num):
        self.child = child # A sub-iterator that we delegate to
        self.num = num     # How many more values we want to grab from it, max
    
    def hasNext(self):
        return self.num > 0 and self.child.hasNext()
        # We must want to return more values AND the child must actually have a value to offer.
    
    def next(self):
        self.num -= 1
        return self.child.next()

the `list` call below worked implicitly by calling the `__iter__` method of `Iterator`, which is called in any context where Python wants an iterator (in the built-in sense) but has been handed a value that isn't one yet.

In [5]:
list(FirstN(AllPositiveIntegers(), 5))

[1, 2, 3, 4, 5]

## Replicating List Comprehensions

How do list comprehensions really work? Python translates them into generators automatically!

First, let's make the built-in `range` function

In [6]:
class Range(Iterator):
    def __init__(self, lower, upper):
        self.lower = lower
        self.upper = upper
    
    def hasNext(self):
        return self.lower < self.upper

    def next(self):
        result = self.lower
        self.lower += 1
        return result

In [7]:
list(Range(3, 6))

[3, 4, 5]

**Composed Iterator**

Let's also make a *composed* iterator which takes in an iterator `iter1` and a function `iter2` that creates another interator. Hence, we loop through each element in `iter1`, then call `iter2` on each element of `iter1`. Note that this is a nested loop.

In [8]:
class Composed(Iterator):
    def __init__(self, iter1, iter2):
        self.iter1 = iter1
        self.iter2 = iter2
        self.specific_iter2 = None # This attribute holds the most recent iterator
                                   # created by the function iter2.
    
    def hasNext(self):
        # It is a bit tricky to tell when there is a next item.
        # We might need to "fast-forward" through many values produced by iter1,
        # if each one corresponds to an empty iter2.
        # As a nice side effect, we stash away the next value to return, if there is one.
        while True:
            if self.specific_iter2 and self.specific_iter2.hasNext():
                return True
        
            if self.iter1.hasNext():
                #keep going to next loop (advancing iter1) until specic_iter2 hasNext
                self.specific_iter2 = self.iter2(self.iter1.next())
            else:
                return False
    
    def next(self):
        if self.hasNext():
            return self.specific_iter2.next()

In [9]:
list(Composed(Range(0, 4), lambda n: Range(n, n+2)))

[0, 1, 1, 2, 2, 3, 3, 4]

**Singleton Iterator**

This yields just one value which we return immediately

In [10]:
class Singleton(Iterator):
    def __init__(self, value):
        self.value = value     # Save to return when asked.
        #this means we cannot run next on this item multiple times
        self.unclaimed = True  # Remember whether we were already asked.
    
    def hasNext(self):
        return self.unclaimed

    def next(self):
        #claim (mark this value as used before) this item
        self.unclaimed = False
        return self.value

In [11]:
list(Composed(Range(0, 4), lambda n: Singleton(n*n)))

[0, 1, 4, 9]

Some examples of using `Composed` and `Singleton`.

Simple List Comprehension:

In [12]:
def map_iterator(func, iter):
    return Composed(iter, lambda x: Singleton(func(x)))

# Equivalent to: [x * 2 for x in range(5)]
doubled = map_iterator(lambda x: x * 2, Range(0, 5))
result = list(doubled)
print(result)  # Output: [0, 2, 4, 6, 8]

[0, 2, 4, 6, 8]


Nested List Comprehension:

In [13]:
# Equivalent to: [(x, y) for x in range(3) for y in range(2)]
nested = Composed(Range(0, 3), 
                  lambda x: Composed(Range(0, 2), 
                                     lambda y: Singleton((x, y))))
result = list(nested)
print(result)  # Output: [(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)]

[(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)]


List Comprehension with Conditional:

In [14]:
# Equivalent to: [x if x % 2 == 0 else 'odd' for x in range(5)]
conditional = map_iterator(lambda x: x if x % 2 == 0 else 'odd', Range(0, 5))
result = list(conditional)
print(result)  # Output: [0, 'odd', 2, 'odd', 4]

[0, 'odd', 2, 'odd', 4]


### Native List Comprehension for Generators

We can also use list comprehension for generators, but instead of storing elements of a list in memory, this creates generator objects. The synatax is `(operation for item in generator/iterable)`

In [15]:
# generator expression
generator_exp = (i * 5 for i in range(5) if i%2==0)

for i in generator_exp:
    print(i)

0
10
20
